In [1]:
import mlflow

mlflow.set_tracking_uri("file:///Users/chloe/PycharmProjects/bnb/bnb_repo/mlruns")
print("Tracking URI:", mlflow.get_tracking_uri())

from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = client.get_experiment_by_name("airbnb_pricing_models_1").experiment_id

# Find best model (lowest RMSE in $)
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.rmse_log ASC"],  # Optional: can remove to get all
    max_results=1
)

if runs:
    best_run = runs[0]
    print("Best run ID:", best_run.info.run_id)
    print("Best RMSE ($):", best_run.data.metrics["rmse_log"])

    # Register the model
    model_uri = f"runs:/{best_run.info.run_id}/model"
    mlflow.register_model(model_uri, "airbnb_best_model_1")
    print("✅ Model registered as 'airbnb_best_model_1'")
else:
    print("⚠️ No runs found.")




Successfully registered model 'airbnb_best_model'.
2025/06/12 20:36:13 WARNING mlflow.tracking._model_registry.fluent: Run with id a20cff517011465cb25e929f360a2248 has no artifacts at artifact path 'model', registering model based on models:/m-ea3c5ee443fa43539a1119741614829c instead


Tracking URI: file:///Users/chloe/PycharmProjects/bnb/bnb_repo/mlruns
Best run ID: a20cff517011465cb25e929f360a2248
Best RMSE ($): 0.37477254069062366
✅ Model registered as 'airbnb_best_model'


Created version '1' of model 'airbnb_best_model'.
